# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Vietnamnet)**


Cũng giống như VNExpress - Demo

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
     {
        "url": "https://vietnamnet.vn/khi-gap-van-xui-hay-nhin-3-...",
        "title": "Khi gặp vận xui, hãy nhìn 3 bức ảnh này sẽ thấy cuộc đời ...",
        "description": "Cuộc đời như thế nào tùy thuộc vào góc nhìn của từng ...",
        "content": "Bức tranh 1: Có 3 hay 4 khúc gỗ?\nCâu trả lời là 3 hoặc 4 khúc gỗ ...",
        "metadata": {
            "cat": "ĐỜI SỐNG",
            "subcat": "GIỚI TRẺ",
            "published_date": 1721274540,
            "author": "Phan Hằng"
        },
    },
    ```


# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [ ]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

ERROR: Could not find a version that satisfies the requirement chromium-chromedriver (from versions: none)
ERROR: No matching distribution found for chromium-chromedriver


In [3]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir -p data

In [4]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [13]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [14]:
MAGAZINE_NAME = "vietnamnet"
HOME_PAGE = "https://vietnamnet.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [16]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [17]:
# Chọn and click menu buttons

#### Choose menu button
all_menu = driver.find_element(by=By.CLASS_NAME, value="icon-hamburger")
#### CLick on the menu button
all_menu.click()

In [18]:
# Lấy hết tất cả thể loại từ đây
cats = []

#### CODE
# row_menu = driver.find_elements(by=By.CLASS_NAME, value="header_submenu-content")
sub_menu = driver.find_element(by=By.CLASS_NAME, value="header_submenu-content-list")

cat_menus = sub_menu.find_elements(by=By.TAG_NAME, value="a")

for cat_name in cat_menus:
    cat = cat_name.get_attribute("title").strip()
    href = cat_name.get_attribute("href").strip()
    
    # bỏ qua logo-premium, logo-htvn
    if cat == "premium vietnamnet" or cat == "Hành trình Việt Nam":
        continue
    else:
        cats.append({"cat_names": cat, "url": href})
####

In [19]:
cats, len(cats)

([{'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'},
  {'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_names': 'Dân tộc và Tôn giáo',
   'url': 'https://vietnamnet.vn/dan-toc-ton-giao'},
  {'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_names': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_names': 'Văn hóa - Giải trí',
   'url': 'https://vietnamnet.vn/van-hoa-giai-tri'},
  {'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_names': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'},
  {'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_names': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_names': 'Du lịch', 'url': 

In [20]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [21]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 30 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vietnamnet_url.json"

EXCLUDING_CATEGORIES = ["Giảm nghèo bền vững", "English", "Toàn văn", "Đính chính"] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm



In [22]:
driver = webdriver.Chrome(options=chrome_options)

In [25]:
cats[2]['url']

'https://vietnamnet.vn/kinh-doanh'

In [23]:
# Vào thử một url
driver.get(cats[2]['url'])

In [26]:
# Get all titles item

#### CODE
title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
####

In [27]:
# Lấy đường dẫn

#### CODE
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
url_new
####

'https://vietnamnet.vn/shb-tren-dinh-lich-su-chung-khoan-shs-dang-ky-ban-20-trieu-co-phieu-2433420.html'

In [28]:
# Tìm next page

#### CODE
page_numbers = driver.find_element(by=By.CLASS_NAME, value="topStory-2nd")

next_page = page_numbers.find_elements(by=By.TAG_NAME, value="a")
next_page = next_page[6].get_attribute("href")
print(next_page)
####

https://vietnamnet.vn/gia-vang-hom-nay-18-8-2025-sjc-can-moc-125-trieu-dong-luong-2432985.html


In [85]:
driver.close()

#### Chạy thật

In [86]:
driver = webdriver.Chrome(options=chrome_options)

In [87]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    # Get all item-news class item
    all_urls = set()
    url = category_url
    
    #### CODE HERE
    while len(all_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
        for title in title_news:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    all_urls.add(url_new)
                    crawled_urls.add(url_new)
            
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                continue
        
        next_pages = driver.find_element(by=By.CLASS_NAME, value="pagination__list")
        url = next_pages.find_elements(by=By.TAG_NAME, value="a")[6].get_attribute("href")
    ####

    return all_urls

In [92]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat['cat_names']
    url = cat['url']
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/thoi-su/chinh-tri'}.
You are at {'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'}.
You are at {'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'}.
You are at {'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'}.
You are at {'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'}.
You are at {'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'}.
You are at {'cat_names': 'Giải trí', 'url': 'https://vietnamnet.vn/giai-tri'}.
You are at {'cat_names': 'Văn hóa', 'url': 'https://vietnamnet.vn/van-hoa'}.
You are at {'cat_names': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'}.
You are at {'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'}.
You are at {'cat_names': 'Thông tin và Truyền thông', 'url': 'https://vietnamnet.vn/thong-tin-truyen-thong'}.
You are at {'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'}.
You are a

In [95]:
len(crawled_urls)

2180

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [29]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vietnamnet_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data\\vietnamnet"

!mkdir -p $DATA_FOLDER_OUTPUT

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [ ]:
# Đọc file url

with open(FILE_URL_PATH, "r", encoding="utf-8") as fIn:
    url_data = json.load(fIn)

len(url_data)

FileNotFoundError: [Errno 2] No such file or directory: 'data/vietnamnet_url.json'

#### Chạy thử nghiệm

In [31]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
    "https://vietnamnet.vn/bat-giam-nguyen-thu-truong-bo-tai-nguyen-va-moi-truong-nguyen-linh-ngoc-2304574.html",
    "https://vietnamnet.vn/ky-su-cong-nghe-ve-que-nuoi-oc-thu-nhe-lai-nua-ty-moi-nam-2304196.html",
    "https://vietnamnet.vn/nsnd-thanh-hoa-va-nsnd-le-dung-hai-nguoi-thay-dac-biet-cua-nsut-viet-hoan-2304214.html",
    #author in last
    "https://vietnamnet.vn/doi-giay-cu-mon-giua-gia-lanh-chau-au-cua-tong-bi-thu-nguyen-phu-trong-2303988.html",
    #class maincontent, table ignore
    "https://vietnamnet.vn/lo-dien-cao-toc-dep-bac-nhat-dat-nuoc-chay-xuyen-rung-ngap-man-2303643.html",
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[2]

In [32]:
driver.get(SAMPLE_ARTICLE_URL)

In [33]:
# Tìm kiếm title

#### CODE HERE
driver.find_element(by=By.CSS_SELECTOR, value="h1.content-detail-title").text
####


'Hai người thầy đặc biệt của NSƯT Việt Hoàn'

In [34]:
# Tìm kiếm description

#### CODE HERE
driver.find_element(by=By.CLASS_NAME, value="sm-sapo-mb-0").text
####


'Trong hành trình nghệ thuật của mình, NSƯT Việt Hoàn may mắn được chịu ảnh hưởng sâu sắc từ hai nghệ sĩ tài năng: NSND Thanh Hoa và NSND Lê Dung.'

In [39]:
# Thu thập thể loại

#### CODE HERE
lis_cat = []
sub_cat = driver.find_element(by=By.CLASS_NAME, value="sm-show-time")
lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
for cat in lis_cats:
    category = cat.get_attribute("title")
    if len(category):
        lis_cat.append(category)
lis_cat
####

['Văn hóa - Giải trí',
 'Chính trị',
 'Thời sự',
 'Kinh doanh',
 'Dân tộc và Tôn giáo',
 'Thể thao',
 'Giáo dục',
 'Thế giới',
 'Đời sống',
 'Văn hóa - Giải trí',
 'Sức khỏe',
 'Công nghệ',
 'Pháp luật',
 'Xe',
 'Bất động sản',
 'Du lịch',
 'Bạn đọc',
 'Thế giới sao']

In [372]:
#Thu thập ngày
#### CODE HERE
publisd_date = driver.find_element(by=By.CLASS_NAME, value="bread-crumb-detail__time").text
publisd_date
####


'thứ hai, 22/07/2024 - 08:16'

In [35]:
# Tìm kiếm contents
#### CODE HERE
article = driver.find_element(by=By.CLASS_NAME, value="main-content")
children = article.find_elements(by=By.XPATH, value="./*")
####

In [36]:
# Thu thập contents
contents = []
author = "Unknown"
is_slide_show = False

for idx, child in enumerate(children):
    text = child.text.strip()
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

        #### CODE HERE
        contents.append(text)
        ####

    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        #### CODE HERE
        if len(text) > 0:
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
        ####

    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [37]:
contents

['Sinh năm 1966 tại Thái Bình, Việt Hoàn lớn lên trong một gia đình có truyền thống nghệ thuật cải lương. Tài năng âm nhạc của anh đã sớm bộc lộ và được ghi nhận khi chỉ ở tuổi 18, anh đã đoạt giải Nhất cuộc thi đơn ca toàn tỉnh Thái Bình năm 1985.',
 'Sự nghiệp của Việt Hoàn là một chuỗi những thành công liên tiếp, từ việc giành được các huy chương trong các hội diễn sân khấu đến giải đặc biệt Tiếng hát làng Sen năm 1990. Năm 1997 đánh dấu bước ngoặt quan trọng khi anh thi đỗ vào khoa Thanh nhạc của Nhạc viện Hà Nội.',
 'Trong hành trình nghệ thuật của mình, Việt Hoàn may mắn được chịu ảnh hưởng sâu sắc từ hai nghệ sĩ tài năng: NSND Thanh Hoa và NSND Lê Dung. NSND Thanh Hoa, người mà Việt Hoàn coi là tượng đài âm nhạc, đã truyền cảm hứng cho anh từ khi còn nhỏ. Việt Hoàn từng chia sẻ về những kỷ niệm đáng nhớ khi theo dõi các buổi biểu diễn của NSND Thanh Hoa, thậm chí phải "chui qua khe hở ở nhà vệ sinh" để được nghe chị hát khi không có tiền mua vé.',
 'NSND Lê Dung, người thầy trực

In [376]:
# Thu thập tác giả
is_author_maybe_last = False
authors = []
try:
    #### CODE HERE
    name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
    author_names = name.find_elements(by=By.CLASS_NAME, value="name")
    for author_name in author_names:
        author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
        authors.append(author)
    ####
except NoSuchElementException:
    is_author_maybe_last = True

# Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
if is_author_maybe_last:
    if len(contents):
        last_content = contents[-1]
        if last_content.find('-') != -1 and last_content.find('-') <= 30:
            authors = last_content[:(last_content.find('-')-1)]
        else:
            authors = "Unknown"

In [377]:
authors

['Văn Hào']

In [378]:
driver.close()

#### Chạy thật

In [392]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    #### CODE HERE
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.content-detail-title").text
    ####

    # Thu thập description
    #### CODE HERE
    description = driver.find_element(by=By.CLASS_NAME, value="sm-sapo-mb-0").text
    ####

    # Thu thập thể loại
    #### CODE HERE
    lis_cat = []
    sub_cat = driver.find_element(by=By.CLASS_NAME, value="sm-show-time")
    lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
    for cat in lis_cats:
        category = cat.get_attribute("title")
        if len(category):
            lis_cat.append(category)
    
    main_cat = lis_cat[0]
    if len(lis_cat) > 1:
        sub_cat = lis_cat[1]
    else:
        sub_cat = "None"
    ####


    # Thu thập published date
    #### CODE HERE
    published_date = driver.find_element(by=By.CLASS_NAME, value="bread-crumb-detail__time").text
    ####


    # Thu thập content bài báo
    #### CODE HERE
    # Locate phần viết content
    article = driver.find_element(by=By.CLASS_NAME, value="main-content")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")
    
    # Thu thập contents
    contents = []
    author = "Unknown"
    is_slide_show = False

    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

            #### CODE HERE
            contents.append(text)
            ####

        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            #### CODE HERE
            if len(text) > 0:
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)
            ####

        elif child.tag_name == "table": # Do nothing rightnow
            pass
    ####

    # Thu thập tác giả
    #### CODE HERE
    is_author_maybe_last = False
    authors = []
    try:
        #### CODE HERE
        name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
        author_names = name.find_elements(by=By.CLASS_NAME, value="name")
        for author_name in author_names:
            author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
            authors.append(author)
        ####
    except NoSuchElementException:
        is_author_maybe_last = True

    # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
    if is_author_maybe_last:
        if len(contents):
            last_content = contents[-1]
            if last_content.find('-') != -1 and last_content.find('-') <= 30:
                authors = last_content[:(last_content.find('-')-1)]
            else:
                authors = "Unknown"


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": published_date,
            "author": author
        }
    }


In [393]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding="utf-8") as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

# Bug will be emagazines and videos, we ignore them as well

Thu thập dữ liệu thể loại Chính trị ..
Thu thập dữ liệu thể loại Thời sự ..
Thu thập dữ liệu thể loại Kinh doanh ..
Thu thập dữ liệu thể loại Thể thao ..
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Giáo dục ..
Thu thập dữ liệu thể loại Giải trí ..
Bug at url: https://vietnamnet.vn/my-nhan-cao-1-72m-hoc-thac-si-kiem-tien-tu-nam-16-tuoi-thi-miss-grand-vietnam-2306665.html, with NoSuchElementException
Thu thập dữ liệu thể loại Văn hóa ..
Thu thập dữ liệu thể loại Đời sống ..
Thu thập dữ liệu thể loại Sức khỏe ..
Thu thập dữ liệu thể loại Thông tin và Truyền thông ..
Thu thập dữ liệu thể loại Pháp luật ..
Thu thập dữ liệu thể loại Ô tô xe máy ..
Thu thập dữ liệu thể loại Bất động sản ..
Thu thập dữ liệu thể loại Du lịch ..
Thu thập dữ liệu thể loại Bạn đọc ..
Thu thập dữ liệu thể loại Tuần Việt Nam ..
Thu thập dữ liệu thể loại Công nghiệp hỗ trợ ..
Thu thập dữ liệu thể loại Bảo vệ người tiêu dùng ..
Thu thập dữ liệu thể loại Thị trường tiêu dùng ..
Thu thập dữ liệu thể 

In [394]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://vietnamnet.vn/bac-lieu-phat-huy-quyen-lam-chu-cua-nguoi-dan-trong-xay-dung-nong-thon-moi-2305220.html',
 'title': 'Bạc Liêu: Phát huy quyền làm chủ của người dân trong xây dựng nông thôn mới',
 'description': 'Tỉnh Bạc Liêu luôn chú trọng phát huy quyền làm chủ của nhân dân, lấy ý kiến về sự hài lòng của người dân làm thước đo trong xây dựng nông thôn mới.',
 'content': 'Với phương châm xây dựng nông thôn mới nâng cao, nông thôn mới kiểu mẫu phải đảm bảo thực chất, không chạy theo thành tích, tỉnh Bạc Liêu luôn chú trọng phát huy quyền làm chủ của nhân dân, lấy ý kiến về sự hài lòng của người dân làm thước đo trong xây dựng nông thôn mới.\nTheo đó, Ban Thường trực Ủy ban Mặt trận Tổ quốc Việt Nam các huyện, thị xã, thành phố trong tỉnh đã chủ trì, phối hợp với các tổ chức chính trị - xã hội, các xã để tiến hành phát phiếu lấy ý kiến sự hài lòng của người dân trên địa bàn về kết quả xây dựng xã đạt chuẩn nông thôn mới nâng cao, nông thôn mới kiểu mẫu.\nThông qua việc lấ

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB